In [27]:
import os

In [28]:
os.chdir('c:\\Users\\Archana\\Desktop\\test\\assignmen_solution\\')

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('artifacts/data_ingestion/employee_data.csv', on_bad_lines='skip')


In [31]:
df.head()

,employee_id,age,gender,marital_status,salary,employment_type,region,has_dependents,tenure_years,enrolled
0,10001,60,Female,Single,55122.97,Part-time,West,No,1.5,0
1,10002,50,Female,Single,89549.66,Full-time,West,Yes,12.8,1
2,10003,36,Male,Divorced,74145.66,Part-time,Midwest,No,3.8,0
3,10004,64,Female,Married,53877.83,Full-time,Northeast,No,3.3,0
4,10005,29,Male,Single,63404.63,Contract,Midwest,Yes,10.0,0


In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE : str
    unzip_data_dir: Path
    all_schema:dict

In [33]:
!pip install scikit-learn

In [34]:
from src.ml_proj.constants import *
from  src.ml_proj.utils.common import read_yaml, create_directories

In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os
from src.ml_proj import logger
import pickle

In [36]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )
        return data_validation_config


In [37]:
class DataPreprocessing:
    def __init__(self, df: pd.DataFrame, config: DataValidationConfig):
        self.config = config
        self.df = df
        # self.encoder = None  # To store the fitted LabelEncoder
        # self.scaler = None   # To store the fitted StandardScaler


    def encode_categorical_columns(self):
        try:

            if 'employee_id' in self.df.columns:
                self.df.drop(columns=['employee_id'], inplace=True)
                logger.info("Dropped Employee ID column.")
                
            categorical_cols = self.df.select_dtypes(include=['object', 'category']).columns.tolist()
            encoder = LabelEncoder()

            for col in categorical_cols:
                self.df[col] = encoder.fit_transform(self.df[col])

                encoder_save_path = os.path.join(self.config.root_dir, f"{col}_encoder.pkl")
            
           
                with open(encoder_save_path, "wb") as f:
                    pickle.dump(encoder, f)

            logger.info("Categorical columns encoded successfully.")

            # Save the encoder as .pkl file



            logger.info(f"LabelEncoder saved at: {encoder_save_path}")
        except Exception as e:
            logger.error(f"Error encoding categorical columns: {e}")
            raise e

    def scale_features(self, features: list):
        
        try:
            scaler = StandardScaler()
            self.df[features] = scaler.fit_transform(self.df[features])

            logger.info(f"Features {features} scaled successfully.")

            # Save the scaler as .pkl file
            scaler_save_path = os.path.join(self.config.root_dir, "scaler.pkl")
            
            with open(scaler_save_path, "wb") as f:
                pickle.dump(scaler, f)

            # joblib.dump(self.scaler, scaler_save_path)
            logger.info(f"StandardScaler saved at: {scaler_save_path}")
        except Exception as e:
            logger.error(f"Error scaling features: {e}")
            raise e

    def save_preprocessed_data(self, save_path):
        
        try:
            # Ensure the directory exists before saving
            save_dir = os.path.dirname(save_path)
            os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

            # Save the preprocessed CSV
            self.df.to_csv(save_path, index=False)

            logger.info(f"Preprocessed data successfully saved at: {save_path}")
            print(f"✅ File saved at: {save_path}")  # Debugging output
        except Exception as e:
            logger.error(f"Error saving preprocessed data: {e}")
            raise e

In [38]:
class DataValidation:
    def __init__(self, config=DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [39]:
try:
    # Initialize Configuration Manager
    config = ConfigurationManager()

    data_validation_config = config.get_data_validation_config()

    # Load dataset
    df = pd.read_csv(data_validation_config.unzip_data_dir)
    print(df.shape)
    # Data preprocessing
    data_preprocessor = DataPreprocessing(df,data_validation_config)

    # Drop Employee ID column
    # data_preprocessor.drop_employee_id()

    # Encode categorical columns
    data_preprocessor.encode_categorical_columns()

    # Scale numerical features
    features_to_scale = ["age", "salary", "tenure_years"]
    data_preprocessor.scale_features(features_to_scale)

    # Data validation
    data_validation = DataValidation(config=data_validation_config)
    validation_passed = data_validation.validate_all_columns()

# Save the preprocessed file in the validation folder
    preprocessed_save_path = os.path.join(config.config.data_validation.validation_dir, "preprocessed_data.csv")
    data_preprocessor.save_preprocessed_data(preprocessed_save_path)

except Exception as e:
    logger.error(f"Error in processing: {e}")
    raise e

[2025-03-27 17:23:33,044: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 17:23:33,044: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 17:23:33,051: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-27 17:23:33,051: INFO: common: created directory at: artifacts]
[2025-03-27 17:23:33,055: INFO: common: created directory at: artifacts/data_validation]
(10000, 10)
[2025-03-27 17:23:33,069: INFO: 3517047002: Dropped Employee ID column.]
[2025-03-27 17:23:33,081: INFO: 3517047002: Categorical columns encoded successfully.]
[2025-03-27 17:23:33,081: INFO: 3517047002: LabelEncoder saved at: artifacts/data_validation\has_dependents_encoder.pkl]
[2025-03-27 17:23:33,081: INFO: 3517047002: Features ['age', 'salary', 'tenure_years'] scaled successfully.]
[2025-03-27 17:23:33,081: INFO: 3517047002: StandardScaler saved at: artifacts/data_validation\scaler.pkl]
[2025-03-27 17:23:33,142: INFO: 3517047002: Preprocessed data successf